In [1]:
import numpy as np
import pandas as pd
import time
from datetime import timedelta
from IPython.display import clear_output
import ipywidgets as widgets

import sys
sys.path.append("../..")
from utils import utils, postprocessing_utils as post
from fine_tuning import fine_tuning

In [2]:
import logging
logging.getLogger("transformers").setLevel(logging.ERROR)

In [3]:
data_path = "../../data/sentiment/"
short_model_name = input("Short model name: ")
experiment = input("Experiment: ")
task = "sentiment"
print("\n")
checkpoints_path = "E:/TFM_CCIL/checkpoints_score_limit/"

# Target score
M = post.Metrics(results_dir="../../results/",
                 experiment="acl",
                 short_model_name=short_model_name,
                 task=task,
                 metric="Macro_F1")
df = M.within_score()
score_limit = np.min(df["Within-Score"])
print("Score limit:", score_limit, "\n")

batches_per_eval = 1

# Model parameters
max_length = 256
batch_size = 8
learning_rate = 2e-5
epochs = 30
num_labels = 2


training_lang = fine_tuning.get_global_training_state(data_path, short_model_name, experiment, checkpoints_path)

Short model name: mbert
Experiment: acl


Score limit: 0.6984357361715852 

Training language:   German 

Already trained:     Slovak     Greek    Vietnamese  Finnish  Korean    Turkish  Hebrew    Maltese
Spanish              Norwegian  Chinese  Thai        Basque   Japanese  Arabic   Algerian

Not yet trained:     Cantonese  Indonesian

Cannot train:      



In [4]:
scores = widgets.Output()
scores

Output()

In [ ]:
start = time.time()

while training_lang is not None:
    trainer = fine_tuning.LimitTrainer(score_limit, training_lang, data_path, task, short_model_name)
    trainer.build_model(max_length, batch_size, learning_rate, epochs, num_labels, eval_batch_size=64)
    trainer.setup_checkpoint(checkpoints_path)
    trainer.prepare_data()
    
    dev_score = trainer.train(batches_per_eval=batches_per_eval)
    with scores:
        print("{}: {}".format(utils.code_to_name[training_lang], dev_score))
    trainer.model = None # Free memory
    time.sleep(5)
        
    clear_output()
    total_elapsed = time.time() - start
    print("Total time elapsed:", timedelta(seconds=np.round(total_elapsed)), "\n")
    training_lang = fine_tuning.get_global_training_state(data_path, short_model_name, experiment, checkpoints_path)

Total time elapsed: 0:58:36 

Training language:   Cantonese 

Already trained:     Spanish  Norwegian  Chinese     Thai     Basque  Japanese  Arabic  Algerian
German               Slovak   Greek      Vietnamese  Finnish  Korean  Turkish   Hebrew  Maltese 

Not yet trained:     Indonesian

Cannot train:      

Successfully built bert-base-multilingual-cased
Final file: E:/TFM_CCIL/checkpoints_score_limit/zh_yue/bert-base-multilingual-cased_sentiment.hdf5
Epoch 0
-------
70/70 [==============================] - 78s 1s/step
Dev Score: 0.46182197158048993
70/70 [==============================] - 79s 1s/step
Dev Score: 0.4592529711375212
70/70 [==============================] - 79s 1s/step
Dev Score: 0.4592529711375212
70/70 [==============================] - 79s 1s/step
Dev Score: 0.4592529711375212
70/70 [==============================] - 80s 1s/step
Dev Score: 0.4592529711375212
70/70 [==============================] - 79s 1s/step
Dev Score: 0.4592529711375212
70/70 [===================

70/70 [==============================] - 81s 1s/step
Dev Score: 0.4592529711375212
70/70 [==============================] - 80s 1s/step
Dev Score: 0.4592529711375212
70/70 [==============================] - 80s 1s/step
Dev Score: 0.4592529711375212
70/70 [==============================] - 80s 1s/step
Dev Score: 0.4592529711375212
70/70 [==============================] - 80s 1s/step
Dev Score: 0.4592529711375212
70/70 [==============================] - 81s 1s/step
Dev Score: 0.4592529711375212
70/70 [==============================] - 80s 1s/step
Dev Score: 0.4592529711375212
70/70 [==============================] - 83s 1s/step
Dev Score: 0.4592529711375212
70/70 [==============================] - 80s 1s/step
Dev Score: 0.4592529711375212
70/70 [==============================] - 80s 1s/step
Dev Score: 0.4592529711375212
70/70 [==============================] - 80s 1s/step
Dev Score: 0.4592529711375212
70/70 [==============================] - 81s 1s/step
Dev Score: 0.4592529711375212
70/7